In [100]:
import pandas as pd
import geopandas as gpd

dtype_1 = {"w_geocode":str}

dtype_2 = {"GEOID20":str}

wac_ma = pd.read_csv("ma_wac_S000_JT00_2021.csv/ma_wac_S000_JT00_2021.csv",dtype=dtype_1)

mass_tract_geom =gpd.read_file("../tract_geom/tl_2020_25_tabblock20/tl_2020_25_tabblock20.shp",dtype=dtype_2).to_crs("EPSG:26986")


In [101]:
mass_tract_geom.crs

<Projected CRS: EPSG:26986>
Name: NAD83 / Massachusetts Mainland
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - Massachusetts onshore - counties of Barnstable; Berkshire; Bristol; Essex; Franklin; Hampden; Hampshire; Middlesex; Norfolk; Plymouth; Suffolk; Worcester.
- bounds: (-73.5, 41.46, -69.86, 42.89)
Coordinate Operation:
- name: SPCS83 Massachusetts Mainland zone (meters)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [102]:
wac_ma_jobs = wac_ma[["w_geocode","C000"]]

In [103]:
wac_ma_jobs_geom=wac_ma_jobs.merge(mass_tract_geom[["GEOID20","geometry"]],right_on="GEOID20",left_on="w_geocode",how="left")
wac_ma_jobs_geom.drop(columns="w_geocode",inplace=True)
wac_ma_jobs_geom = gpd.GeoDataFrame(wac_ma_jobs_geom,geometry=wac_ma_jobs_geom["geometry"])
wac_ma_jobs_geom = wac_ma_jobs_geom.set_crs("EPSG:26986")

In [104]:
wac_ma_jobs_geom["sqft"] = ((wac_ma_jobs_geom['geometry'].area) * 10.7639)

In [105]:
wac_ma_jobs_geom["jobspersqft"] = wac_ma_jobs_geom["C000"] / wac_ma_jobs_geom["sqft"]
wac_ma_jobs_geom = wac_ma_jobs_geom.to_crs("EPSG:4326")

In [106]:
station_areas = pd.read_csv("../stations_routes_f23_geoms_nonoverlapping.csv")

station_areas= gpd.GeoDataFrame(station_areas,geometry=gpd.GeoSeries.from_wkt(station_areas["walk_poly_wkt"]))
station_areas = station_areas.set_crs("EPSG:4326")

In [107]:
wac_ma_jobs_geom['centroid'] = wac_ma_jobs_geom.centroid

# Step 2: Create a new GeoDataFrame for centroids with the same CRS
centroids_gdf = gpd.GeoDataFrame(wac_ma_jobs_geom, geometry='centroid', crs=wac_ma_jobs_geom.crs)

c:\Users\Ted Banken\.conda\envs\sal\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [108]:
stations_w_jobs = gpd.sjoin(station_areas,centroids_gdf, predicate='contains')

In [109]:
stations_w_jobs

,route_id,parent_station,walk_poly_wkt,geometry_left,index_right,C000,GEOID20,geometry_right,sqft,jobspersqft
0,Blue,place-wondl,"POLYGON ((-70.99617 42.41447, -70.99505 42.418...","POLYGON ((-70.99617 42.41447, -70.99505 42.418...",43636,49,250251705041003,"POLYGON ((-70.99348 42.40884, -70.99333 42.409...",1.861831e+06,0.000026
0,Blue,place-wondl,"POLYGON ((-70.99617 42.41447, -70.99505 42.418...","POLYGON ((-70.99617 42.41447, -70.99505 42.418...",43643,41,250251705042001,"POLYGON ((-71.00017 42.40962, -70.99989 42.410...",2.094576e+06,0.000020
0,Blue,place-wondl,"POLYGON ((-70.99617 42.41447, -70.99505 42.418...","POLYGON ((-70.99617 42.41447, -70.99505 42.418...",43644,1,250251705042002,"POLYGON ((-70.99455 42.41647, -70.99437 42.416...",1.441687e+05,0.000007
0,Blue,place-wondl,"POLYGON ((-70.99617 42.41447, -70.99505 42.418...","POLYGON ((-70.99617 42.41447, -70.99505 42.418...",43638,39,250251705041007,"POLYGON ((-70.98975 42.41492, -70.98953 42.415...",3.162193e+05,0.000123
0,Blue,place-wondl,"POLYGON ((-70.99617 42.41447, -70.99505 42.418...","POLYGON ((-70.99617 42.41447, -70.99505 42.418...",43645,1,250251705042005,"POLYGON ((-70.99333 42.41744, -70.99252 42.417...",1.125461e+05,0.000009
...,...,...,...,...,...,...,...,...,...,...
131,Red,place-brntn,"POLYGON ((-71.00607 42.20921, -71.00457 42.209...","POLYGON ((-71.00607 42.20921, -71.00457 42.209...",34457,31,250214198002020,"POLYGON ((-71.00416 42.20733, -71.00386 42.207...",5.824420e+05,0.000053
131,Red,place-brntn,"POLYGON ((-71.00607 42.20921, -71.00457 42.209...","POLYGON ((-71.00607 42.20921, -71.00457 42.209...",34444,5,250214198002003,"POLYGON ((-71.00005 42.20914, -70.99940 42.209...",2.247216e+06,0.000002
131,Red,place-brntn,"POLYGON ((-71.00607 42.20921, -71.00457 42.209...","POLYGON ((-71.00607 42.20921, -71.00457 42.209...",34281,11,250214191004020,"POLYGON ((-71.00431 42.21186, -71.00376 42.211...",1.104258e+06,0.000010
131,Red,place-brntn,"POLYGON ((-71.00607 42.20921, -71.00457 42.209...","POLYGON ((-71.00607 42.20921, -71.00457 42.209...",34443,151,250214198002001,"POLYGON ((-71.00030 42.21112, -71.00027 42.211...",1.046876e+05,0.001442


In [117]:
stations_wac = stations_w_jobs.groupby(["route_id","parent_station"]).mean()[["jobspersqft"]].reset_index().rename(columns={"jobspersqft":"mean_jobs_sqft"})

stations_wac["tot_jobs"] = stations_w_jobs.groupby(["route_id","parent_station"]).sum()[["C000"]].reset_index().rename(columns={"C000":"tot_jobs"})["tot_jobs"]

In [118]:
stations_wac

,route_id,parent_station,mean_jobs_sqft,tot_jobs
0,Blue,place-aport,0.000113,625
1,Blue,place-aqucl,0.001605,5169
2,Blue,place-bmmnl,0.000056,232
3,Blue,place-bomnl,0.003926,55847
4,Blue,place-gover,0.006872,27638
...,...,...,...,...
127,Red,place-qnctr,0.001126,10691
128,Red,place-shmnl,0.000182,1726
129,Red,place-smmnl,0.000188,1427
130,Red,place-sstat,0.011649,85483


In [116]:
stations_wac.to_csv("stations_routes_2021_wac_agg.csv")